In [1]:
from OPqkdsim.channel import *
from OPqkdsim.measure import *
from OPqkdsim.polarization import *
from OPqkdsim.source import *
from OPqkdsim.timeline import *
from OPqkdsim.random import *
from OPqkdsim.classpostpro import *

In [2]:
# Initialize Timeline

timeline = Timeline()
shots=1000
# Define bit lists and modulation lists
rng = CRNG(method="crypto")
alice_bit_list = rng.generate_bits(shots)
alice_modulation_list = rng.generate_basis(shots)
bob_modulation_list = rng.generate_basis(shots)
bob_bit_list = [] 

In [3]:
# Initialize Alice Components

laser = DFBLaser(timeline, shots=shots)
pc1 = PolarizationController(timeline)  # Alice's polarization control
pm1 = AlicePolarizationModulator(timeline, modulation_list=alice_modulation_list, bit_list=alice_bit_list)
attenuator = VariableOpticalAttenuator(timeline, attenuation_dB=60)  # Attenuates to single-photon level # why 60db ?

# Quantum channel

channel = QuantumChannel(timeline)  


In [4]:
# Initialize Bob Components

pc2 = PolarizationController(timeline, compensate=True)  # Bob's first polarization control
pm2 = BobPolarizationModulator(timeline, modulation_list=bob_modulation_list)
pc3 = PolarizationController(timeline)  # Final polarization correction

# Create Detectors

detector_HV = SinglePhotonDetector(timeline, name="Detector_H", bit_list=bob_bit_list)
#detector_V = SinglePhotonDetector(timeline, name="Detector_V", bit_list=bob_bit_list)
detector_D = SinglePhotonDetector(timeline, name="Detector_D+", bit_list=bob_bit_list)
#detector_D_minus = SinglePhotonDetector(timeline, name="Detector_D-", bit_list=bob_bit_list)

# Create PBS

pbs = PolarizationBeamSplitter(timeline, detector_HV, detector_D, bob_modulation_list)


In [5]:
# Connect Components Using Timeline’s Publish-Subscribe Model

timeline.subscribe(laser, pc1.process_signal)
timeline.subscribe(pc1, pm1.process_signal)
timeline.subscribe(pm1, attenuator.process_signal)
timeline.subscribe(attenuator, channel.propagate_signal)
timeline.subscribe(channel, pc2.process_signal)
timeline.subscribe(pc2, pm2.process_signal)
timeline.subscribe(pm2, pc3.process_signal)
timeline.subscribe(pc3, pbs.process_signal)
timeline.subscribe(pbs, detector_HV.detect_photon)
timeline.subscribe(pbs, detector_D.detect_photon)

In [6]:
# Start Simulation by Emitting Light from the Laser
laser.initialize_laser()
timeline.execute_events()

[1.74049e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:1000
[1.74049e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:999
[1.74049e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74049e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:998
[1.74049e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74049e+09 s] Phase Modulated: Basis=0, Bit=1, φ=3.14159 rad
[1.74049e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:997
[1.74049e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74049e+09 s] Phase Modulated: Basis=1, Bit=1, φ=-1.57080 rad
[1.74049e+09 s] VOA applied 60.00000 dB attenuation. New Power: 1.20000e-13 W
[1.74049e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W, Shots left:996
[1.74049e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74049e+09 s] Phase Modulated: Basis=0, Bit=1, φ=3.14159 rad
[1.74049e+09 s] VOA applied 60.00000 dB a

In [7]:
# matching_bits = []
# for alice_basis, bob_basis, alice_bit, bob_bit in zip(alice_modulation_list, bob_modulation_list, alice_bit_list, bob_bit_list):
#     if alice_basis == bob_basis:  # Matching bases
#         matching_bits.append(alice_bit)

# # Display Results
# print("\n--- BB84 Protocol Results ---")
# print(f"Alice's Bits: {alice_bit_list}")
# print(f"Alice's Bases: {alice_modulation_list}")
# print(f"Bob's Bases: {bob_modulation_list}")
# print(f"Bob's Received Bits: {bob_bit_list}")
# print(f"Final Key (Matching Basis): {matching_bits}")

In [8]:
# shiftedkey = []
# skc = SiftedKey(timeline, np.array(bob_bit_list), np.array(bob_modulation_list), shiftedkey)
# classical_channel = ClassicalChannel(timeline,skc.receive_alice_bases)


# timeline.subscribe(classical_channel, skc.receive_alice_bases)
# message = {"basis": np.array(alice_modulation_list)}
# classical_channel.transmit_message(message)
# timeline.execute_events()
# print(shiftedkey)

In [9]:
shiftedkey = []
skc = SiftedKey(timeline, np.array(alice_bit_list), np.array(alice_modulation_list), shiftedkey)
skc.receive_bob_bases(None,bob_modulation_list)
print(shiftedkey)

Alice computed sifted key of length 506.
[1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0

In [10]:
lpdcer = LDPCReconciliation(shiftedkey)
pb = lpdcer.compute_parity_bits()
print(pb)

[1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0
 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 1
 0 1 0 0 1 1 0 0 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1]


In [11]:
correct_key = lpdcer.correct_errors(shiftedkey)
print(len(correct_key))
print(correct_key)

506
[0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 1
 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0
 0 0 1 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1
 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 0
 0 0 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0
 1 1 1 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 1
 0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0
 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1
 1 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 0 1
 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0
 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0
 1 1 1 1 0 0 1 1 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1
 1 0 0 1 0 1 1 0 1 1 

In [12]:
sl = len(shiftedkey)
qber = QBERCalculator(sl,int(0.1*sl))
rawqber = qber.calculate_qber(np.array(alice_bit_list),np.array(bob_bit_list))
erqber = qber.calculate_statistical_error()
approxqber = qber.calculate_approximated_qber()

approximated_qber = 0.46 ± 0.07048404074682439


In [13]:
pramp = UhashPA()
hash_key = pramp.amplify(correct_key)
print(hash_key)

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.